Datos de Hubble

In [37]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math
import random
import heapq

In [38]:
doc2 = open("C:/Users/antoc/Downloads/PortableGit/practica2024/hub_data.txt")
lineas2 = doc2.readlines()

# Divide cada línea en columnas usando espacio como separador
datos_separados2 = [linea.strip().split(' ') for linea in lineas2]

datos= []
for fila in datos_separados2:
    datos.append(fila)

z = []
for i in range(len(datos)):
      a = float(datos[i][0])
      z.append(a)
z =np.array(z)

H = []
for i in range(len(datos)):
   if datos[i][1] == '':
      a = float(datos[i][2])
      H.append(a)
   else:
      a = float(datos[i][1])
      H.append(a)
H = np.array(H)

desv = []
for i in range(len(datos)):
    a = float(datos[i][2])
    desv.append(a)


In [39]:
def exp(x):
    return np.exp(x)
def ln(x):
    return np.log(x)
def poli1(x):
    return x 
def poli2(x):
    return x**2
def poli3(x):
    return x**3

grammar = [exp, ln, poli1, poli2, poli3]
operaciones = ['+', '-', '*', '/']

#A partir de las funciones ingresadas inicialmente, calcula el chi_2
def chi_2_valores(f):
    return ((H - f(z))/desv)**2

def chi_2_f(f):
    chi_2_val = np.zeros_like(f)
    chi_2 = np.zeros_like(f)
    for i in range(len(f)):
        chi_2_val[i] = chi_2_valores(f[i])
        for j in range(len(chi_2_val)):
            chi_2[j] = np.sum(chi_2_val[j])
    
    return chi_2

print(chi_2_f(grammar))
chi_inic = chi_2_f(grammar)

[4157.01591713639 4430.910574765109 4318.165617110693 4300.926784511239
 4239.109972002022]


### Combinar
Se combinan las funciones iniciales con operaciones entre ellas hasta que sean 50 funciones iniciales.

In [40]:
def juntar_elementos(funciones, operaciones, longitud_final=50):
    final = []
    while len(final) < longitud_final * 3:
        random_funcion = random.choice(funciones)
        random_operador = random.choice(operaciones)

        final.append(random_funcion)
        final.append(random_operador)
        final.append(random_funcion)

    # Crear una lista de sublistas para representar las filas
    resultado_final = [final[i:i+3] for i in range(0, len(final), 3)]

    return resultado_final

resultado_final = juntar_elementos(grammar, operaciones)

#for fila in resultado_final:
    #print(fila)


Convertir la lista a función

In [44]:
def convertir_lista_a_funcion1(lista):
    def nueva_funcion(x):
        resultado = lista[0]
        for i in range(1, len(lista), 2):
            operador = lista[i]
            valor = lista[i + 1]

            if operador == '+':
                resultado += valor
            elif operador == '-':
                resultado -= valor
            elif operador == '*':
                resultado *= valor
            elif operador == '/':
                resultado /= valor

        return resultado

    return nueva_funcion

# Ejemplo de uso
mi_lista = [2, '+', 3, '*', 5, '/', 2]

# Convertir la lista en una función
mi_funcion = convertir_lista_a_funcion1(mi_lista)

# Evaluar la función en un valor específico
resultado = mi_funcion(3)
print("Resultado de la función en x=0:", resultado)

def convertir_lista_a_funcion(lista):
    def nueva_funcion(x):
        for i in range(len(lista)-1):
            if callable(lista[i]):
                if lista[i+1] != ' ':
                    func1 = lista[i]
                    func2 = lista[i+2]
            elif isinstance(lista[i], str):
                if lista[i] == '+':
                    f = func1(x) + func2(x)
                elif lista[i] == '-':
                    f = func1(x) - func2(x)
                elif lista[i] == '*':
                    f = func1(x) * func2(x)
                elif lista[i] == '/':
                    f = func1(x) / func2(x)

        return f

    return nueva_funcion
                
lista_prueba = [poli1, '+', poli1]
funcion_prueba= convertir_lista_a_funcion(lista_prueba)
print(funcion_prueba(1))


    

Resultado de la función en x=0: 12.5
2


In [42]:
def operar_funciones(func1, func2, operaciones):
    operadores = {'+': lambda x: func1(x) + func2(x),
                  '-': lambda x: func1(x) - func2(x),
                  '*': lambda x: func1(x) * func2(x),
                  '/': lambda x: func1(x) / func2(x) if func2(x) != 0 else np.inf if func1(x) > 0 else -np.inf}

    def combinacion(x):
        resultados = []
        for operacion in operaciones:
            resultados.append(operadores[operacion](x))

        return resultados
    return combinacion


def juntar_funciones(funcion_externa, funcion_interna):
    def funcion_combinada(x):
        resultado_interno = funcion_interna(x)
        resultado_externo = funcion_externa(resultado_interno)
        return resultado_externo
    
    return funcion_combinada

def cambiar_operadores(f, op_cambiar, op_nueva):
    nueva_expresion = f.__code__.co_code.decode('utf-8')
    nueva_expresion = nueva_expresion.replace(op_cambiar, op_nueva)

    def nueva_funcion(x):
        return eval(nueva_expresion, {'x': x})

    return nueva_funcion  # Añadido para devolver la nueva función

# Evaluar la función combinada original
func_combinada = operar_funciones(exp, exp, operaciones)
x = 0
resultados_original = func_combinada(x)
print("Resultados original:", resultados_original)

# Cambiar el operador en la expresión de la función
nueva_funcion = cambiar_operadores(func_combinada, '+', '-')
resultados_nueva = nueva_funcion(x)
print("Resultados con operador cambiado:", resultados_nueva)


Resultados original: [2.0, 0.0, 1.0, 1.0]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x88 in position 4: invalid start byte

In [ ]:
#Se juntan todas las combinaciones entre funciones
combinaciones_iniciales = np.zeros((len(grammar), len(grammar)), dtype=object)
for i in range(len(grammar)):
    for j in range(len(grammar)):
        combinaciones_iniciales[i][j] = funcion_combinada(grammar[i], grammar[j], operaciones)
'''
for row in combinaciones_iniciales:
    for func_combinada in row:
        print(func_combinada(0))  
'''

#Se obtiene chi^2 para todas las combinaciones
print(chi_2_f(combinaciones_iniciales))
chi_combinaciones = chi_2_f(combinaciones_iniciales)

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
# Obtener el índice del elemento aleatorio
#indice_elemento_aleatorio = mi_lista.index(elemento_aleatorio)

In [ ]:
#Cantidad de sobrevivientes inicialmente 
#Todas las funciones iniciales son seleccionadas
tasa_seleccion= 1
cant_sobrevivientes = tasa_seleccion*len(chi_inic)
cant_sobreviv = int(cant_sobrevivientes)
print(cant_sobreviv)


#chi_menores = sorted(chi_inic)[:cant_sobreviv]


9


In [ ]:
#Cantidad de sobrevivientes en el tiempo
tasa_seleccion= 0.2
cant_sobrevivientes = tasa_seleccion*(len(combinaciones_iniciales)*len(combinaciones_iniciales))
cant_sobreviv = int(cant_sobrevivientes)
print(cant_sobreviv)


16


In [ ]:
#Para obtener los valores de chi menores y los índices
mi_lista = [5, 2, 8, 10, 1, 7, 3, 6, 4, 9, 12, 15, 11, 14, 13, 16, 17, 18, 19, 20]

# Número de elementos más pequeños
num_elementos = cant_sobreviv

# Obtener los índices y valores de los elementos más pequeños
indices_valores_menores = sorted(enumerate(mi_lista), key=lambda x: x[1])[:num_elementos]

# Separar los índices y valores en listas distintas
indices_menores, valores_menores = zip(*indices_valores_menores)

# Imprimir los resultados
print("Lista original:", mi_lista)    
print("Índices de los primeros 16 elementos más pequeños:", indices_menores)
print("Valores de los primeros 16 elementos más pequeños:", valores_menores)

Lista original: [5, 2, 8, 10, 1, 7, 3, 6, 4, 9, 12, 15, 11, 14, 13, 16, 17, 18, 19, 20]
Índices de los primeros 16 elementos más pequeños: (4, 1, 6, 8, 0, 7, 5, 2, 9, 3, 12, 10, 14, 13, 11, 15)
Valores de los primeros 16 elementos más pequeños: (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)


In [ ]:
'''
para asignar un peso a cada función
'''

def combinar_funciones(grammar, pesos):
    def funcion_combinada(x):
        resultado = 0.0
        for i, funcion in enumerate(grammar):
            resultado += pesos[i] * funcion(x)
        return resultado
    
    return funcion_combinada

# Asigna pesos inversamente proporcionales a los chi cuadrado
pesos = 1.0 / chi_2_f(grammar)

# Crea la función combinada
funciones_pe = combinar_funciones(grammar, pesos)

# Evalúa la función combinada en un punto
x_evaluado = 1.0
resultado_combinado = funciones_pe(x_evaluado)

print("Resultado combinado en", x_evaluado, "es:", resultado_combinado)

Resultado combinado en 1.0 es: 0.00231059182768764
